In [909]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import math
from geopy.geocoders import Nominatim
import time
import re

In [910]:
# Прочитаем файл с объявлением квартир
df = pd.read_csv('/Users/darinaafanaseva/Desktop/DataScience/project/data/_data.csv')
pd.set_option('display.max_columns', None)

In [911]:
# Создадим отдельную со столбцами 'ID  объявления', 'Метро', 'Адрес'
geo_data = df[['ID  объявления', 'Метро', 'Адрес']]

In [912]:
# Фильтруем значения адреса по городу Москва
geo_data = geo_data[geo_data['Адрес'].str.contains('Москва')]

In [913]:
# Оставляем только Москву, улицу и номер дома (просто для красоты)
for index, _ in geo_data.iterrows():    
    addres = geo_data.loc[index, 'Адрес']    
    geo_data.loc[index, 'Адрес'] = ', '.join(addres.split(',')[:3])

In [914]:
# Убираем всё лишнее в столбце Метро
geo_data['Road_to_metro'] = geo_data['Метро'].str.extract(r'\(([^)]+)')
geo_data[geo_data['Метро'].isna()==False]
df_encoded = pd.get_dummies(geo_data, columns=['Метро'])

In [915]:
# Добавляем метро для Зеленограда и путь до метро на машине
for index, _ in geo_data.iterrows():    
    addres = geo_data.loc[index, 'Адрес']
    if 'еленогр' in addres:
        geo_data.loc[index, 'Метро'] = "Речной вокзал"
        geo_data.loc[index, 'Road_to_metro'] = "30 минут на машине"

In [916]:
# Рассчитываем отдельное время пешком и преобразуем путь на машине в путь пешком
for index, _ in geo_data.iterrows():    
    road = geo_data.loc[index, 'Road_to_metro']        
    t = re.findall(r'\d+', str(road))    
    if len(t) > 0:
        if 'ешком' in road:            
            geo_data.loc[index, 'время пешком'] = int(t[0])
        else:
            geo_data.loc[index, 'время пешком'] = int(t[0]) * 12
    else:
        geo_data.loc[index, 'время пешком'] = 0   

In [917]:
# Очистка от пустых значений метро
geo_data_full = geo_data[geo_data['Метро'].isna()==False]

In [918]:
# Выделяем отдельное название от станций метро
for index, _ in geo_data_full.iterrows():    
    metro = geo_data_full.loc[index, 'Метро']    
    add_split = metro.split("(")
    geo_data_full.loc[index, 'Метро'] = add_split[0]

res_df = geo_data_full

In [919]:
# Удаление временных столбцов
res_df = res_df.drop(columns=['Road_to_metro'])

In [920]:
# Переименовываем
res_df.rename(columns={'ID  объявления': 'ID'}, inplace=True)

In [921]:
# Столбец парковка сведен к значениям 0/1 (отсутствие/наличие парковки), тк команда решила обобщить признак до двух значений и посчитала, что вид парковки не показал бы значительного колебания в цене аренды
df['Parking']=df['Парковка'].apply(lambda x: 0 if pd.isna(x) else 1)
df.drop(columns=['Парковка'], inplace=True)


In [922]:
# Столбец "Дополнительно" разделен на 11 столбцов, в каждом из которых проставлены 0/1, отражающие наличие того или иного атрибута в квартире
df['Furniture_in_the_rooms']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Мебель в комнатах' in x else 0)
df['Kitchen_furniture']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Мебель на кухне' in x else 0)
df['Bath']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Ванна' in x else 0)
df['Washing_machine']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Стиральная машина' in x else 0)
df['Air_conditioner']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Кондиционер' in x else 0)
df['Dishwasher']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Посудомоечная машина' in x else 0)
df['TV']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Телевизор' in x else 0)
df['Internet']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Интернет' in x else 0)
df['Telephone']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Телефон' in x else 0)
df['Shower_cabin']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Душевая кабина' in x else 0)
df['Fridge']= df['Дополнительно'].astype(str).apply(lambda x: 1 if 'Холодильник' in x else 0)
df.drop(columns=['Дополнительно'], inplace=True)

In [923]:
# Добавляем дополнительные столбцы относительно наличия балконов и отлельный столбец с количеством
df[['Балкон1', 'Балкон2']] = df['Балкон'].str.split(', ', expand=True)
df['value1'] = df['Балкон1'].str.extract(r'(\d+)')
df['value2'] = df['Балкон2'].str.extract(r'(\d+)')
df['value1'] = pd.to_numeric(df['value1'])
df['value2'] = pd.to_numeric(df['value2'])
df['Baclony'] = df['value1'] + df['value2']
df['Baclony'] = df['Baclony'].fillna(df['value1'])
df['Baclony']=df['Baclony'].fillna(0)


In [924]:
# Оцифровываем туалеты
df[['Совмещенный', 'Раздельный']] = df['Санузел'].str.split(', ', expand=True)
df['Совмещенный'] = df['Совмещенный'].str.extract(r'(\d+)')
df['Раздельный'] = df['Раздельный'].str.extract(r'(\d+)')
df['Combined'] = pd.to_numeric(df['Совмещенный']).fillna(0)
df['Separate'] = pd.to_numeric(df['Раздельный']).fillna(0)
df['Bathroom'] = df['Combined'] + df['Separate']

In [925]:
# Оцифровываем лифты
df[['Пасс', 'Груз']] = df['Лифт'].str.split(', ', expand=True)
df['Пасс'] = df['Пасс'].str.extract(r'(\d+)')
df['Груз'] = df['Груз'].str.extract(r'(\d+)')
df['Passenger'] = pd.to_numeric(df['Пасс']).fillna(0)
df['Cargo'] = pd.to_numeric(df['Груз']).fillna(0)
df['Elevator'] = df['Passenger'] + df['Cargo']

In [926]:
# Взяли общую площадь помещения
df['Square,m2'] = df['Площадь, м2'].astype(str).str.split('/').str[0].astype(float)

In [927]:
# Привели значения в столбце "Rooms" к числовому значению, за исключением Nan
df['Rooms'] = df['Количество комнат'].astype(str).str.split(',').str[0]
df['Rooms'] = pd.to_numeric(df['Rooms'], errors='coerce')
df['Rooms'] = df['Rooms'].astype('Int64')

In [928]:
# Нашли среднюю площадь комнаты относительно тех данных, которые у нас есть
df['Square,m2'] = pd.to_numeric(df['Square,m2'], errors='coerce')
df_notna = df.dropna(subset=['Rooms', 'Square,m2'])
average_room_size = (df_notna['Square,m2'] / df_notna['Rooms']).mean()

In [929]:
# Заполнили пропуски в столбце 'Rooms', основываясь на общую площадь квартиры и среднюю площадь одной комнаты по датасету
df['Rooms'] = df.apply(
    lambda row: np.round(row['Square,m2'] / average_room_size) if pd.isnull(row['Rooms']) else row['Rooms'],
    axis=1
).astype('Int64')

In [930]:
# Заполнили пропуски в столбце 'Renovation' на значение 'Не указано'
df['Renovation'] = df['Ремонт'].fillna('Не указано')

In [931]:
# Создали столбцы 'Children','Animal', в которых отменитили относительно каждого объявления опции
df['Children'] = np.where(df['Можно с детьми/животными'].str.contains('Можно с детьми', na=False), 1, 0)
df['Animal'] = np.where(df['Можно с детьми/животными'].str.contains('Можно с животными', na=False), 1, 0)

In [932]:
# Посчитали среднюю высоту потолков в объявлениях с известными данными и заполнили пропуски средним значением
mean_height = df['Высота потолков, м'].mean().round(1)
df['Height,m'] = df['Высота потолков, м'].fillna(mean_height)
df = df.rename(columns={
 'ID  объявления': 'ID',
 'Адрес': 'Address',  
})


In [933]:
# Оцифровываем дома 
df[['Floor', 'House']] = df['Дом'].str.split(',', expand=True)
df[['Floor_flat', 'Floor_all']] = df['Floor'].str.split('/', expand=True)
df['House'] = df['House'].fillna('Неизвестный')
df['Floor_flat'] = df['Floor_flat'].astype('Int64')
df['Floor_all'] = df['Floor_all'].astype('Int64')

In [934]:
# Разделяем столбец с ценой на отдельные с ценой, с суммой залога и с ком услугами
price = df['Цена'].dropna()
rent_price = price.apply(lambda x: x.split('/')[0])
df['Price'] = rent_price.apply(lambda x: float(x.split(' ')[0]))
df['Deposit'] = df['Цена'].str.extract(r'Залог - (\d+) руб')
df['Public_services']= df['Цена'].apply(lambda x: 1 if "Коммунальные услуги включены" in x else 0)

In [935]:
# ЗАЧЕМ ТО ДОБАВЛЯЕМ НОВУЮ ПЕРЕМЕННУЮ, НАМ ТАК БЫЛО УДОБНЕЕ
df_1 = df[['ID', 'Rooms','Address','Square,m2','House','Floor_all','Floor_flat','Цена',
           'Renovation','Baclony','Bathroom','Children','Animal','Height,m','Elevator',
           'Parking','Furniture_in_the_rooms','Kitchen_furniture','Bath','Washing_machine',
           'Air_conditioner','Dishwasher','TV','Internet','Telephone','Shower_cabin','Fridge', 'Price',	'Deposit',	'Public_services']]
df_1

,ID,Rooms,Address,"Square,m2",House,Floor_all,Floor_flat,Цена,Renovation,Baclony,Bathroom,Children,Animal,"Height,m",Elevator,Parking,Furniture_in_the_rooms,Kitchen_furniture,Bath,Washing_machine,Air_conditioner,Dishwasher,TV,Internet,Telephone,Shower_cabin,Fridge,Price,Deposit,Public_services
0,271271157,4,"Москва, улица Новый Арбат, 27",200.0,Монолитный,16,5,"500000.0 руб./ За месяц, Залог - 500000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес",Дизайнерский,0.0,0.0,1,1,3.00,5.0,1,1,1,1,1,1,1,1,1,1,1,1,500000.0,500000,1
1,271634126,4,"Москва, улица Новый Арбат, 27",198.0,Монолитно-кирпичный,16,5,"500000.0 руб./ За месяц, Залог - 500000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес",Дизайнерский,0.0,3.0,1,0,3.50,2.0,1,1,1,1,1,1,1,1,1,0,1,1,500000.0,500000,1
2,271173086,4,"Москва, улица Новый Арбат, 27",200.0,Неизвестный,16,5,"500000.0 руб./ За месяц, Залог - 500000 руб., Срок аренды - Длительный, Предоплата 1 мес",Евроремонт,0.0,3.0,1,0,3.20,1.0,1,1,1,1,1,1,1,1,1,1,1,1,500000.0,500000,0
3,272197456,4,"Москва, переулок Плотников, 21С1",170.0,Неизвестный,6,5,"400000.0 руб./ За месяц, Залог - 400000 руб., Срок аренды - Длительный, Предоплата 1 мес",Евроремонт,0.0,3.0,0,1,3.20,1.0,1,1,1,1,1,1,1,1,1,1,1,1,400000.0,400000,0
4,273614615,2,"Москва, улица Новый Арбат, 15",58.0,Панельный,26,12,"225000.0 руб./ За месяц, Залог - 225000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес",Евроремонт,0.0,2.0,0,0,3.90,2.0,0,1,1,1,1,0,1,1,1,0,1,1,225000.0,225000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 км до МКАД), ш. Сколковское (8 км до МКАД)",35.0,Неизвестный,14,10,"42000.0 руб./ За месяц, Залог - 42000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес",Евроремонт,1.0,1.0,0,0,3.00,2.0,0,1,1,1,1,1,0,0,1,0,0,1,42000.0,42000,1
23364,274654844,1,"Москва, Производственная улица, 8к1, ш. Боровское (5 км до МКАД), ш. Киевское (7 км до МКАД)",38.7,Монолитный,18,5,"45000.0 руб./ За месяц, Залог - 45000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес",Евроремонт,1.0,1.0,0,0,3.00,2.0,0,1,1,1,1,0,0,1,0,0,0,1,45000.0,45000,1
23365,268679909,2,"Москва, Боровский проезд, 11",43.1,Кирпичный,5,5,"50000.0 руб./ За месяц, Залог - 50000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес",Дизайнерский,1.0,1.0,1,0,3.00,0.0,0,0,1,0,1,1,0,0,1,0,1,1,50000.0,50000,1
23366,274807525,2,"Москва, улица Богданова, 6к1, ш. Боровское (3 км до МКАД), ш. Сколковское (8 км до МКАД)",52.5,Монолитный,23,8,"55000.0 руб./ За месяц, Залог - 50000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 2 мес",Евроремонт,1.0,2.0,0,0,2.65,3.0,1,1,1,1,1,1,1,0,0,0,0,1,55000.0,50000,1


In [936]:
# Смотрим значения в столбце Ремонта
df_1['Renovation'].value_counts()

Renovation
Косметический    8499
Евроремонт       8470
Дизайнерский     3474
Не указано       2755
Без ремонта       170
Name: count, dtype: int64

In [937]:
# Делаем one-hot encoding
df_1['Reno_design'] = df_1['Renovation'].apply(lambda x: x == 'Дизайнерский')
df_1['Reno_euro'] = df_1['Renovation'].apply(lambda x: x == 'Евроремонт')
df_1['Reno_cosmetic'] = df_1['Renovation'].apply(lambda x: x == 'Косметический')
df_1['Reno_not_specified'] = df_1['Renovation'].apply(lambda x: x == 'Не указано')
df_1['Reno_no_renovation'] = df_1['Renovation'].apply(lambda x: x == 'Без ремонта')

/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/580381199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Reno_design'] = df_1['Renovation'].apply(lambda x: x == 'Дизайнерский')
/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/580381199.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Reno_euro'] = df_1['Renovation'].apply(lambda x: x == 'Евроремонт')
/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/580381199.py:4: SettingWithCopyWarning: 
A v

In [938]:
# Заполняем колонки
df_1['Reno_design'] = df_1['Reno_design'].astype(int)
df_1['Reno_euro'] = df_1['Reno_euro'].astype(int)
df_1['Reno_cosmetic'] = df_1['Reno_cosmetic'].astype(int)
df_1['Reno_not_specified'] = df_1['Reno_not_specified'].astype(int)
df_1['Reno_no_renovation'] = df_1['Reno_no_renovation'].astype(int)


/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/2965785984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Reno_design'] = df_1['Reno_design'].astype(int)
/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/2965785984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Reno_euro'] = df_1['Reno_euro'].astype(int)
/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/2965785984.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [939]:
# Столбец "Дом" разделен на 7 столбцов, в каждом из которых проставлены 0/1 
df_1['Panel']= df_1['House'].astype(str).apply(lambda x: 1 if 'Панельный' in x else 0)
df_1['Monolithic']= df_1['House'].astype(str).apply(lambda x: 1 if 'Монолитный' in x else 0)
df_1['Brick']= df_1['House'].astype(str).apply(lambda x: 1 if 'Кирпичный' in x else 0)
df_1['Block']= df_1['House'].astype(str).apply(lambda x: 1 if 'Блочный' in x else 0)
df_1['Monolithic_brick']= df_1['House'].astype(str).apply(lambda x: 1 if 'Монолитно-кирпичный' in x else 0)
df_1['Stalin']= df_1['House'].astype(str).apply(lambda x: 1 if 'Сталинский' in x else 0)
df_1['Old_fond']= df_1['House'].astype(str).apply(lambda x: 1 if 'старый фонд' in x else 0)

/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/991121481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Panel']= df_1['House'].astype(str).apply(lambda x: 1 if 'Панельный' in x else 0)
/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/991121481.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['Monolithic']= df_1['House'].astype(str).apply(lambda x: 1 if 'Монолитный' in x else 0)
/var/folders/f_/z8w1jqr12zs_vk1rjhlpn3740000gn/T/ipykernel_5412/991121481.py:4: 

In [940]:
# Мёрджим всё
df_merged = pd.merge(df_1, res_df[['ID', 'Метро', 'Адрес', 'время пешком']], on='ID', how='left')


In [941]:
df_merged.rename(columns={'Метро': 'metro', 'Адрес': 'adress', 'время пешком': 'walk'}, inplace=True)

In [942]:
df_merged = df_merged.drop('Address', axis=1)
df_merged = df_merged.drop('House', axis=1)

In [943]:
df_merged.drop('Цена', axis=1, inplace=True)
df_merged

,ID,Rooms,"Square,m2",Floor_all,Floor_flat,Renovation,Baclony,Bathroom,Children,Animal,"Height,m",Elevator,Parking,Furniture_in_the_rooms,Kitchen_furniture,Bath,Washing_machine,Air_conditioner,Dishwasher,TV,Internet,Telephone,Shower_cabin,Fridge,Price,Deposit,Public_services,Reno_design,Reno_euro,Reno_cosmetic,Reno_not_specified,Reno_no_renovation,Panel,Monolithic,Brick,Block,Monolithic_brick,Stalin,Old_fond,metro,adress,walk
0,271271157,4,200.0,16,5,Дизайнерский,0.0,0.0,1,1,3.00,5.0,1,1,1,1,1,1,1,1,1,1,1,1,500000.0,500000,1,1,0,0,0,0,0,1,0,0,0,0,0,м. Смоленская,"Москва, улица Новый Арбат, 27",9.0
1,271634126,4,198.0,16,5,Дизайнерский,0.0,3.0,1,0,3.50,2.0,1,1,1,1,1,1,1,1,1,0,1,1,500000.0,500000,1,1,0,0,0,0,0,0,0,0,1,0,0,м. Смоленская,"Москва, улица Новый Арбат, 27",8.0
2,271173086,4,200.0,16,5,Евроремонт,0.0,3.0,1,0,3.20,1.0,1,1,1,1,1,1,1,1,1,1,1,1,500000.0,500000,0,0,1,0,0,0,0,0,0,0,0,0,0,м. Смоленская,"Москва, улица Новый Арбат, 27",7.0
3,272197456,4,170.0,6,5,Евроремонт,0.0,3.0,0,1,3.20,1.0,1,1,1,1,1,1,1,1,1,1,1,1,400000.0,400000,0,0,1,0,0,0,0,0,0,0,0,0,0,м. Смоленская,"Москва, переулок Плотников, 21С1",3.0
4,273614615,2,58.0,26,12,Евроремонт,0.0,2.0,0,0,3.90,2.0,0,1,1,1,1,0,1,1,1,0,1,1,225000.0,225000,1,0,1,0,0,0,1,0,0,0,0,0,0,м. Арбатская,"Москва, улица Новый Арбат, 15",7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23363,215565511,1,35.0,14,10,Евроремонт,1.0,1.0,0,0,3.00,2.0,0,1,1,1,1,1,0,0,1,0,0,1,42000.0,42000,1,0,1,0,0,0,0,0,0,0,0,0,0,м. Говорово,"Москва, Боровское шоссе, 2к7",8.0
23364,274654844,1,38.7,18,5,Евроремонт,1.0,1.0,0,0,3.00,2.0,0,1,1,1,1,0,0,1,0,0,0,1,45000.0,45000,1,0,1,0,0,0,0,1,0,0,0,0,0,м. Солнцево,"Москва, Производственная улица, 8к1",7.0
23365,268679909,2,43.1,5,5,Дизайнерский,1.0,1.0,1,0,3.00,0.0,0,0,1,0,1,1,0,0,1,0,1,1,50000.0,50000,1,1,0,0,0,0,0,0,1,0,0,0,0,м. Солнцево,"Москва, Боровский проезд, 11",6.0
23366,274807525,2,52.5,23,8,Евроремонт,1.0,2.0,0,0,2.65,3.0,1,1,1,1,1,1,1,0,0,0,0,1,55000.0,50000,1,0,1,0,0,0,0,1,0,0,0,0,0,м. Солнцево,"Москва, улица Богданова, 6к1",11.0
